# Text Analysis on ferguson data

In [20]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ferguson.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [22]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,0
14,498248415223246848,14090948,The mother of the boy killed in #Ferguson spea...,Sat Aug 09 23:24:13 +0000 2014,0
40,498248648699150336,14090948,Police have brought out the large gear in #Fer...,Sat Aug 09 23:25:09 +0000 2014,0
96,498250100079341568,1921517718,"Omg Wow ""@FLOCKAfierce: WOW RT @AntonioFrench:...",Sat Aug 09 23:30:55 +0000 2014,0
100,498251940997136384,228120405,When you see citizens as enemies... RT @Antoni...,Sat Aug 09 23:38:14 +0000 2014,0
...,...,...,...,...,...
22910,500413818368184321,179732982,"Remarkably, despite the military-style police ...",Fri Aug 15 22:48:46 +0000 2014,1
22929,500417151346696192,2193946021,Americans are 8 times more likely to be killed...,Fri Aug 15 23:02:00 +0000 2014,1
22982,500421926079066112,5513002,Shot ten times.\nUnarmed.\nWhat else is there ...,Fri Aug 15 23:20:59 +0000 2014,1
22996,500422971320963072,470708968,I refuse to call the #Ferguson police chief in...,Fri Aug 15 23:25:08 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [23]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
          
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    text = [stem.stem(i) for i in text]
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [24]:
print("original text: ", df.reply_tweet.iloc[20])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[20]))

original text:  @AntonioFrench @AngieinWAState Cops on speed. Time we cleaned out the police depts. made them do psychological testing. This is a horror.
cleaned text:  cop speed time clean polic dept made psycholog test thi horror


### applied on original data (**df**)

In [25]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [26]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,0,black teenag boy men children stop refer 17 ye...,black teenag boy men children stop refer 17 ye...
1,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,498243332204949504,587926686,@annaxsweat @NeoSoulPol Same thing when #Trayv...,Sat Aug 09 23:04:01 +0000 2014,0,black teenag boy men children stop refer 17 ye...,thing trayvon wa kill zimmerman wannab cop 17y...
2,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,498265524397432832,21461995,@annaxsweat http://t.co/Z7Qm4iaoSR,Sun Aug 10 00:32:13 +0000 2014,0,black teenag boy men children stop refer 17 ye...,
3,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,498266827676741632,21461995,@annaxsweat #StopThugCops We have to take acti...,Sun Aug 10 00:37:23 +0000 2014,0,black teenag boy men children stop refer 17 ye...,stopthugcop take action stop thi insan ferguson
4,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,498272808560889858,2411474370,@annaxsweat @KidFriendlyCamb 17 year olds can ...,Sun Aug 10 01:01:09 +0000 2014,0,black teenag boy men children stop refer 17 ye...,17 year old prosecut adult regardless color so...


### applied on source tweet data (**src_tw_df**) 

In [27]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [28]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [29]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [30]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [31]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw,src_sentiment,reply_sentiment
0,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,498235547685756928,1539100118,Black teenage boys are not men. They are child...,Sat Aug 09 22:33:06 +0000 2014,0,black teenag boy men children stop refer 17 ye...,black teenag boy men children stop refer 17 ye...,Negative,Negative


### applied on source tweet data only 

In [32]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/ferguson/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/ferguson/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/ferguson/senti_source.pdf',dpi=300)

### save data to csv 

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ferguson/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ferguson/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ferguson/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [33]:
import text2emotion as te

In [34]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [39]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emotion']=emo

In [40]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-df.csv',index=False)

In [41]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emotion']=emo1

In [42]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-src.csv',index=False)